In [ ]:
# | default_exp codegen

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from fastcore.test import *
from jinja2 import UndefinedError

import jupyter_black
import tempfile
import nbdev.showdoc as showdoc

jupyter_black.load()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from pathlib import Path
from jinja2 import Environment, BaseLoader, Template
from typing import Optional, Any
from jinja2 import StrictUndefined
from textwrap import dedent
from yaml.parser import ParserError
from black import format_str, FileMode

from sal.loaders import xml_to_data
from sal.core import Data
from sal.frontmatter import FrontMatter
from sal.templates import Renderer, InMemoryTemplateLoader, JinjaTemplateRenderer

# Code Generation

## What code generation means with sal? 

In it's basic form, it will combine xml files converted to `Data` structures, with jinja templates, to render code. Later we will also introduce some frontmatter.

For this, we need a basic structure to work with for generating code. As an example, we'll be working with an hypotetical "model"

In [ ]:
struct: Data = xml_to_data(
    """
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
    """
)

...and the basic templates used with this structure are:

In [ ]:
model = (
    "class {{ name }}Model(models.Model):\n"
    "    {%- for child in children %}\n"
    "    {{ child | render }}\n"
    "    {%- endfor %}\n"
)

field = "{{ name }} = models.{{ type | title }}Field()"

## Code generator I (jinja only)

Now that we can render `jinja2`, we can make a basic code generator

In [ ]:
# | export
class SalBasic:
    def __init__(self, renderer: Optional[Renderer] = None):
        self.renderer = renderer or Renderer()

    def pre_process_data(self, data: Data):
        return data

    def action_default(self, data: Data):
        return self.renderer.process(data)

    def action_to_file(self, data: Data):
        rendered = self.renderer.render(data, template=Renderer.DEFAULT_TEMPLATE)
        to = data.attrs["to"]
        with open(to, "w") as h:
            h.write(rendered)
        return rendered

    def action_black(self, data: Data):
        rendered = self.renderer.render(data, template=Renderer.DEFAULT_TEMPLATE)
        return format_str(rendered, mode=FileMode())

    def process_data(self, data: Data):
        if data.name == "to-file":
            return self.action_to_file(data)
        elif data.name == "black":
            return self.action_black(data)
        elif data.name == "wrapper":
            return [self.process(d) for d in data.children]
        else:
            return self.action_default(data)

    def process(self, data: Data):
        data = self.pre_process_data(data)
        return self.process_data(data)

It's important to note that a parent should be able the trigger the rendering of his children (this enures the recursive nature of the template rendering). Look at the `model` template for an example:

In [ ]:
model = (
    "class {{ name }}Model(models.Model):\n"
    "    {%- for child in children %}\n"
    "    {{ child | render }}\n"
    "    {%- endfor %}\n"
)

field = "{{ name }} = models.{{ type | title }}Field()"

In [ ]:
# | hide
repository = InMemoryTemplateLoader(
    templates={
        "model": model,
        "field": field,
    }
)
template_renderer = Renderer(repository=repository, renderer=JinjaTemplateRenderer())

With this, here's a basic jinja2-based code generator using the hard coded templates:

In [ ]:
sal = SalBasic(template_renderer)
test_eq(
    sal.process(struct.clone()).strip(),
    """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

**todo: document to-file**

**todo: document black**

**todo: document wrapper**

In [ ]:
# | hide
struct2: Data = xml_to_data(
    """
<wrapper>
<wrapper>
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</wrapper>
</wrapper>"""
)

sal = SalBasic(template_renderer)
test_eq(
    sal.process(struct2.clone())[0][0].strip(),
    """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

We are missing one more thing, we need to be able to save the result to a file and we'd like to have that info in the xml and not mess with code to get the job done. So, here's a new struct:

In [ ]:
destination = tempfile.NamedTemporaryFile()

s_file = xml_to_data(
    f"""
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>"""
)


sal = SalBasic(template_renderer)
print(sal.process(s_file.clone()))

class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()


In [ ]:
# | hide

destination = tempfile.NamedTemporaryFile()

s_file = xml_to_data(
    f"""
<wrapper>
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>
</wrapper>"""
)

sal = SalBasic(template_renderer)
result = sal.process(s_file)

with open(destination.name, "r") as h:
    test_eq(
        h.read(),
        """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
    """.strip(),
    )

## Code generator II (jinja + frontmatter)

To make this even more powerful, we can use `frontmatter` to embed meta data into the templates themself and merge those with the attributes of the node. 

To make it even more powerful, the frontmatter can contain any attribute from the struct so it needs to be extracted in a raw formar, rendered and then extracted. But first, we need new templates..

In [ ]:
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""

In [ ]:
fm = FrontMatter().get_raw_frontmatter(model)
test_eq(fm, 'reference:  "sigla-{{ node.attrs.name | lower }}-model"')

**todo**

In [ ]:
# | export


class FrontMatterMixin:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.frontmatter_handler = FrontMatter()

    def get_template(self, data: Data, frontmatter=False):
        template = super().get_template(data)
        if not frontmatter:
            template = self.frontmatter_handler.get_content(template)
        else:
            template = self.frontmatter_handler.get_raw_frontmatter(template)
        return template


class FrontMatterInMemoryTemplateLoader(FrontMatterMixin, InMemoryTemplateLoader):
    pass

**todo**

In [ ]:
# | export
class Sal(SalBasic):
    def get_frontmatter_attributes_for_data(self, template: str, data: Data) -> dict:
        rendered = self.renderer.render(data, template)
        parsed = self.renderer.repository.frontmatter_handler.parse(rendered)
        return parsed

    def pre_process_data(self, data: Data):
        for d, _ in data:
            if d.name in ["to-file", "black", "wrapper"]:
                continue

            # load template
            template = self.renderer.repository.get_template(d.name, frontmatter=True)
            # handle front matter
            new_attributes = self.get_frontmatter_attributes_for_data(template, d)

            # update attributes
            d.attrs.update(new_attributes)
        return data

In [ ]:
repository = FrontMatterInMemoryTemplateLoader(
    templates={
        "model": model,
        "field": field,
    }
)
template_renderer2 = Renderer(repository=repository, renderer=JinjaTemplateRenderer())

sal = Sal(template_renderer2)
test_eq(
    sal.process(struct.clone()).strip(),
    """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

In [ ]:
# | hide

destination = tempfile.NamedTemporaryFile()
s_file = xml_to_data(
    f"""
    <wrapper>
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>
    </wrapper>
"""
)

# sal = Sal(template_renderer2)
# sal.pre_process_data(s_file.clone())

In [ ]:
sal = Sal(template_renderer2)
sal.process(s_file)

with open(destination.name, "r") as h:
    test_eq(
        h.read().strip(),
        """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
    """.strip(),
    )

In [ ]:
xml = xml_to_data(
    """
<black>
    <W>
        <a/>
        <a/>
        <b/>
    </W>
</black>
"""
)


w = """
---
---
{%- for i in node|imports|sum(None, [])|unique %}
{{ i }}
{%- endfor %}

class W:
    {%- for child in children %}
    {{ child | render }}
    {%- endfor %}
    
"""


a = """
---
imports: 
    - from AAA import A
---
a = AAA( )
"""
b = """
---
imports: 
    - from BBB import B
---
b = BBB()
"""

repository = FrontMatterInMemoryTemplateLoader(
    templates={
        "W": w,
        "a": a,
        "b": b,
    }
)


def imports(data: Data):
    imports_ = [d.attrs.get("imports") for d, _ in data]
    imports_ = [d for d in imports_ if d]
    return imports_


template_renderer2 = Renderer(
    repository=repository,
    renderer=JinjaTemplateRenderer(),
    filters={"imports": imports},
)


sal = Sal(template_renderer2)
res = sal.process(xml)
test_eq(
    res.strip(),
    """
from AAA import A
from BBB import B


class W:
    a = AAA()
    a = AAA()
    b = BBB()
""".strip(),
)

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()